In [2]:
import pandas as pd
import numpy as np
import math

In [3]:
df = pd.read_csv("qbrdv2.csv")

In [4]:
df.head()

,experiment,run,type,module,name,attrname,attrvalue,value
0,Base,0,config,NaN,NaN,*.client[*].KEYSPACE_SIZE,50,NaN
1,Base,0,config,NaN,NaN,*.client[*].PUT_INTERVAL,2000ms,NaN
2,Base,0,config,NaN,NaN,**.CLIENTS_NUMBER,1,NaN
3,Base,0,scalar,QBRD.client[0],# of aborted put,NaN,NaN,0.0
4,Base,0,scalar,QBRD.client[0],# of put requests,NaN,NaN,1800.0


In [5]:
res = pd.DataFrame(columns=['experiment','run','clientsNumber','putInterval','keyspaceSize','avg'])

In [6]:
res.head()

,experiment,run,clientsNumber,putInterval,keyspaceSize,avg


In [7]:
df_base = df[df['experiment'] == 'Base']

In [8]:
df_base.shape[0]

3618

In [35]:
df_rowa = df[df['experiment'] == 'ROWA']

In [36]:
n_clients_base = df_rowa[df_rowa['attrname'] == '**.CLIENTS_NUMBER'][['run','attrvalue']].rename({'attrvalue':'n_clients'}, axis=1)

In [37]:
df_rowa = df_rowa.merge(n_clients_base, on='run', how='left').drop_duplicates()

In [38]:
df_rowa.head()

,experiment,run,type,module,name,attrname,attrvalue,value,n_clients
0,ROWA,0,config,NaN,NaN,*.client[*].PUT_INTERVAL,2000ms,NaN,1
1,ROWA,0,config,NaN,NaN,**.CLIENTS_NUMBER,1,NaN,1
2,ROWA,0,config,NaN,NaN,*.client[*].KEYSPACE_SIZE,50,NaN,1
3,ROWA,0,scalar,QBRD.client[0],# of committed put,NaN,NaN,1800.0,1
4,ROWA,0,scalar,QBRD.client[0],# of get responses,NaN,NaN,0.0,1


In [39]:
put_interval_base = df_rowa[df_rowa['attrname'] == '*.client[*].PUT_INTERVAL'][['run','attrvalue']].rename({'attrvalue':'putInterval'}, axis=1)

In [40]:
df_rowa = df_rowa.merge(put_interval_base, on='run', how='left').drop_duplicates()

In [41]:
df_rowa.head()

,experiment,run,type,module,name,attrname,attrvalue,value,n_clients,putInterval
0,ROWA,0,config,NaN,NaN,*.client[*].PUT_INTERVAL,2000ms,NaN,1,2000ms
1,ROWA,0,config,NaN,NaN,**.CLIENTS_NUMBER,1,NaN,1,2000ms
2,ROWA,0,config,NaN,NaN,*.client[*].KEYSPACE_SIZE,50,NaN,1,2000ms
3,ROWA,0,scalar,QBRD.client[0],# of committed put,NaN,NaN,1800.0,1,2000ms
4,ROWA,0,scalar,QBRD.client[0],# of get responses,NaN,NaN,0.0,1,2000ms


In [42]:
keyspace_base = df_rowa[df_rowa['attrname'] == '*.client[*].KEYSPACE_SIZE'][['run','attrvalue']].rename({'attrvalue':'keyspaceSize'}, axis=1)

In [43]:
df_rowa = df_rowa.merge(keyspace_base, on='run', how='left').drop_duplicates()

In [44]:
df_rowa.head()

,experiment,run,type,module,name,attrname,attrvalue,value,n_clients,putInterval,keyspaceSize
0,ROWA,0,config,NaN,NaN,*.client[*].PUT_INTERVAL,2000ms,NaN,1,2000ms,50
1,ROWA,0,config,NaN,NaN,**.CLIENTS_NUMBER,1,NaN,1,2000ms,50
2,ROWA,0,config,NaN,NaN,*.client[*].KEYSPACE_SIZE,50,NaN,1,2000ms,50
3,ROWA,0,scalar,QBRD.client[0],# of committed put,NaN,NaN,1800.0,1,2000ms,50
4,ROWA,0,scalar,QBRD.client[0],# of get responses,NaN,NaN,0.0,1,2000ms,50


In [45]:
n_committed_put = df_rowa[df_rowa['name'] == '# of committed put'][['run','value']].rename({'value':'n_committed_put'}, axis=1)

In [46]:
n_committed_put[n_committed_put['run']==58]

,run,n_committed_put
489,58,1635.0
495,58,1625.0


In [47]:
n_committed_put_avg = n_committed_put.groupby('run', as_index=False).mean()

In [48]:
n_committed_put_avg.shape[0]

216

In [49]:
df_rowa = df_rowa.merge(n_committed_put_avg, on='run', how='left').drop_duplicates()

In [50]:
df_rowa.head()

,experiment,run,type,module,name,attrname,attrvalue,value,n_clients,putInterval,keyspaceSize,n_committed_put
0,ROWA,0,config,NaN,NaN,*.client[*].PUT_INTERVAL,2000ms,NaN,1,2000ms,50,1800.0
1,ROWA,0,config,NaN,NaN,**.CLIENTS_NUMBER,1,NaN,1,2000ms,50,1800.0
2,ROWA,0,config,NaN,NaN,*.client[*].KEYSPACE_SIZE,50,NaN,1,2000ms,50,1800.0
3,ROWA,0,scalar,QBRD.client[0],# of committed put,NaN,NaN,1800.0,1,2000ms,50,1800.0
4,ROWA,0,scalar,QBRD.client[0],# of get responses,NaN,NaN,0.0,1,2000ms,50,1800.0


In [51]:
df_rowa.shape[0]

3618

In [52]:
n_put_requests = df_rowa[df_rowa['name'] == '# of put requests'][['run','value']].rename({'value':'n_put_requests'}, axis=1)

In [53]:
n_put_requests_avg = n_put_requests.groupby('run', as_index=False).mean()

In [54]:
df_rowa = df_rowa.merge(n_put_requests_avg, on='run', how='left').drop_duplicates()

In [55]:
df_rowa.head()

,experiment,run,type,module,name,attrname,attrvalue,value,n_clients,putInterval,keyspaceSize,n_committed_put,n_put_requests
0,ROWA,0,config,NaN,NaN,*.client[*].PUT_INTERVAL,2000ms,NaN,1,2000ms,50,1800.0,1800.0
1,ROWA,0,config,NaN,NaN,**.CLIENTS_NUMBER,1,NaN,1,2000ms,50,1800.0,1800.0
2,ROWA,0,config,NaN,NaN,*.client[*].KEYSPACE_SIZE,50,NaN,1,2000ms,50,1800.0,1800.0
3,ROWA,0,scalar,QBRD.client[0],# of committed put,NaN,NaN,1800.0,1,2000ms,50,1800.0,1800.0
4,ROWA,0,scalar,QBRD.client[0],# of get responses,NaN,NaN,0.0,1,2000ms,50,1800.0,1800.0


In [56]:
res['experiment'] = df_rowa['experiment']
res['run'] = df_rowa['run']
res['clientsNumber'] = df_rowa['n_clients']
res['putInterval'] = df_rowa['putInterval']
res['keyspaceSize'] = df_rowa['keyspaceSize']
res['avg'] = df_rowa['n_committed_put'] / df_rowa['n_put_requests']
res['putRequests'] = df_rowa['n_put_requests']
res['avgCommitedPut'] = df_rowa['n_committed_put']


In [57]:
res.drop_duplicates(inplace=True)

In [58]:
res.shape[0]

216

In [59]:
res.head(100)

,experiment,run,clientsNumber,putInterval,keyspaceSize,avg,putRequests,avgCommitedPut
0,ROWA,0,1,2000ms,50,1.000000,1800.0,1800.0
8,ROWA,1,1,2000ms,40,1.000000,1800.0,1800.0
16,ROWA,2,1,2000ms,30,1.000000,1800.0,1800.0
24,ROWA,3,1,2000ms,20,1.000000,1800.0,1800.0
32,ROWA,4,1,2000ms,10,1.000000,1800.0,1800.0
...,...,...,...,...,...,...,...,...
965,ROWA,95,2,200ms,5,0.607889,18000.0,10942.0
978,ROWA,96,2,200ms,3,0.446917,18000.0,8044.5
991,ROWA,97,2,200ms,2,0.313889,18000.0,5650.0
1004,ROWA,98,2,200ms,1,0.053750,18000.0,967.5


In [60]:
res.to_csv('rowa_results.csv', index=False)